<a href="https://colab.research.google.com/github/Inderivator/PlaylistSorter/blob/Playlist-sorter/AI_Playlist_Sorter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Convert music sequences into spectrograms

Padding and Truncating

Add CNN feature encoder

Embedding normalization ?

In [ ]:
# Fonction pour normaliser les embeddings de spectrogrammes
def normalize_spectrogram_embeddings(embeddings):
    mean = tf.reduce_mean(embeddings, axis=-1, keepdims=True)
    std = tf.math.reduce_std(embeddings, axis=-1, keepdims=True)
    normalized_embeddings = (embeddings - mean) / std
    return normalized_embeddings

Positional encoding ?

In [ ]:
# Fonction pour ajouter l'encodage de position aux embeddings
def positional_encoding(max_len, d_model):
    pos = tf.range(max_len, dtype=tf.float32)[:, tf.newaxis]
    i = tf.range(d_model, dtype=tf.float32)[tf.newaxis, :]
    angle_rates = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    angle_rads = pos * angle_rates

    # Utilise une opération sinus/cosinus pour encoder les positions
    sines = tf.math.sin(angle_rads[:, 0::2])
    cosines = tf.math.cos(angle_rads[:, 1::2])

    # Concatène les sinus et cosinus pour former l'encodage de position
    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

Create attention masks ?

In [ ]:
#Example :
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    # Ajoute une dimension supplémentaire pour indiquer la séquence de padding
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)


Define parameters of the encoder

In [ ]:
# Paramètres du modèle
max_len = 100  # Longueur maximale de la séquence
d_model = 256  # Dimension des embeddings

# Création du masque de padding
seq = tf.constant([[1, 2, 3, 0, 0]])
padding_mask = create_padding_mask(seq)

# Ajout de l'encodage de position
pos_encoding = positional_encoding(max_len, d_model)

# Normalisation des embeddings de spectrogrammes
embeddings = tf.random.normal((1, max_len, d_model))
normalized_embeddings = normalize_spectrogram_embeddings(embeddings)

# Définition de l'architecture de l'encodeur
inputs = layers.Input(shape=(max_len, d_model))
encoder_output = layers.MultiHeadAttention(num_heads=8, key_dim=64)(inputs, inputs, attention_mask=padding_mask)
encoder_output = layers.LayerNormalization(epsilon=1e-6)(encoder_output)
encoder_output = layers.Dense(d_model, activation='relu')(encoder_output)

# Création du modèle
encoder_model = tf.keras.Model(inputs=inputs, outputs=encoder_output)

# Affichage de l'architecture du modèle
encoder_model.summary()

Define parameters of the decoder

Build of the complete model

In [ ]:
#Construction du modèle complet
num_layers = 2
num_heads = 4
ff_dim = 256
dropout_rate = 0.2

encoder_model = encoder(
    num_layers=num_layers,
    num_heads=num_heads,
    ff_dim=ff_dim,
    input_shape=(sequence_length, num_features),  # sequence_length et num_features à définir
    dropout_rate=dropout_rate,
)


 Compile the model

In [ ]:
#Compilation du modèle
optimizer = Adam(learning_rate=1e-4)
encoder_model.compile(optimizer=optimizer, loss="mse")  # Utilisez votre fonction de perte appropriée

Train the model

In [ ]:
# Utilisez vos données d'entraînement (X_train, y_train) pour entraîner le modèle
encoder_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Evaluate the model

In [ ]:
# Utilisez des données de validation ou de test (X_val, y_val) pour évaluer la performance du modèle
loss = encoder_model.evaluate(X_val, y_val)


Inference of the model

In [ ]:
# Faites des prédictions sur de nouvelles données (X_test)
predictions = encoder_model.predict(X_test)

Save the model

In [ ]:
encoder_model.save("encoder_model.h5")